# Automação de Consulta de Processos Jurídicos

## Contexto

Este notebook apresenta um processo de automação de preenchimento de um formulário de pesquisa de processos jurídicos de um site fictício. Essencialmente, o processo envolve duas etapas: escolha de um Estado e preenchimento do formulário de busca de processos.

## Guia do Processo

Para executar esta automação, vamos seguir este passo-a-passo:

1. Abrir um navegador Google Chrome

2. Abrir a página de busca (index.html)

3. Selecionar um estado

4. Mudar de aba

5. Preencher o formulário de busca

6. Enviar o formulário

7. Confirmar o alerta após envio do formulário

8. Aguardar resultado da busca

9. Atualizar __status__ do processo na base de processos

10. Exportar planilha com dados atualizados

## Importações

In [1]:
# Importa a biblioteca time
import time

# Importa a biblioteca os
import os

# Importa o pandas
import pandas as pd

# Importa o webdriver
from selenium import webdriver

# Importa o Service para especificarmos o caminho do webdriver
from selenium.webdriver.chrome.service import Service

# Importa o web driver manager para instalarmos o webdriver compatível com a versão do Chrome
from webdriver_manager.chrome import ChromeDriverManager

# Instala o Chrome Driver e fornece o caminho do executável
servico = Service(ChromeDriverManager().install())

# Importa a classe By para localizar elementos
from selenium.webdriver.common.by import By

# Importa o Action Chains para trabalharmos com drop-downs dinâmicos
from selenium.webdriver import ActionChains

# Importa o Alert para trabalharmos com alertas
from selenium.webdriver.common.alert import Alert

# Importa o expected conditions para esperar elementos aparecerem
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Importação da Base de Dados

In [2]:
# Importa a base de dados
df_processos = pd.read_excel('Processos.xlsx')

# Exibe as cinco primeiras linhas do dataframe
df_processos.head()

,Nome,Advogado,Processo,Cidade,Status
0,Márcio Diogo Almada,Rayssa Márcia Castro,PC6592,Distrito Federal,NaN
1,Antonio Renato Fábio Silva,Letícia Valentina Camila da Rocha,EB3792,Rio de Janeiro,NaN
2,Igor Lorenzo Ramos,Maria Fátima da Costa,MM1043,Rio de Janeiro,NaN
3,Iago Thomas Vitor Duarte,Marcela Juliana Vitória Moraes,PC5197,São Paulo,NaN
4,Regina Juliana Carolina Novaes,Tiago Lorenzo Gabriel Alves,PC5030,Distrito Federal,NaN


## Criação do navegador

In [3]:
# Cria um navegdor Google Chrome
navegador = webdriver.Chrome(service=servico)

## Abrir a Página de Buscas

In [4]:
# Caminho deste notebook
caminho_nb = os.getcwd()

# Arquivo que desejamos abrir
pagina_busca_estado = 'index.html'

# Caminho completo da página
caminho_pagina_busca_estado = os.path.join(caminho_nb, pagina_busca_estado)

# Abre a página de busca por estado
navegador.get(caminho_pagina_busca_estado)

# Aba original
aba_original = navegador.window_handles[0]

## Selecionar um Estado e Preencher Formulário

In [5]:
# Percorre cada uma das linhas do dataframe
for linha in df_processos.index: 
    # Volta para a aba original
    navegador.switch_to.window(aba_original)
    
    # Localiza o menu
    menu_estados = navegador.find_element(By.CSS_SELECTOR, '.dropdown-menu .btn')

    # Posiciona o mouse sobre o menu
    ActionChains(navegador).move_to_element(menu_estados).perform()

    # Aguarda 1s
    time.sleep(1)
    
    # Cidade que desejamos selecionar
    cidade = df_processos.loc[linha, 'Cidade']
    
    # Localiza a cidade e clica na opção
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()
    
    # Nova aba
    nova_aba = navegador.window_handles[1]
    
    # Muda para a nova aba
    navegador.switch_to.window(nova_aba)
    
    # Localiza o campo de nome e preenche ele
    navegador.find_element(By.ID, 'nome').send_keys(df_processos.loc[linha, "Nome"])

    # Localiza o campo de advogado e preenche ele
    navegador.find_element(By.ID, 'advogado').send_keys(df_processos.loc[linha, "Advogado"])

    # Localiza o campo número do processo e preenche ele
    navegador.find_element(By.ID, 'numero').send_keys(df_processos.loc[linha, "Processo"])
    
    # Localiza o botão de enviar formulário e clica nele
    navegador.find_element(By.CSS_SELECTOR, '#formulario .btn.registerbtn').click()
    
    # Seleciona o alerta e aceita ele
    alerta = Alert(navegador)

    # Aceita o alerta
    alerta.accept()
    
    # Espera pelo alerta, aguardando 1s a cada iteração
    while True:
        try:
            alerta = navegador.switch_to.alert
            break
        except:
            time.sleep(1)

    # Extrai o texto do alerta
    texto_alerta = alerta.text

    if texto_alerta:
        if 'Processo encontrado' in texto_alerta:
            alerta.accept()
            df_processos.loc[linha, 'Status'] = 'Processo Encontrado' 
        else:
            alerta.accept()
            df_processos.loc[linha, 'Status'] = 'Processo não Encontrado'
    
    # Fecha a aba atual
    navegador.close()

In [6]:
# Fecha o navegador
navegador.quit()

## Dados Atualizados

In [7]:
# Exibe o dataframe
df_processos.head()

,Nome,Advogado,Processo,Cidade,Status
0,Márcio Diogo Almada,Rayssa Márcia Castro,PC6592,Distrito Federal,Processo não Encontrado
1,Antonio Renato Fábio Silva,Letícia Valentina Camila da Rocha,EB3792,Rio de Janeiro,Processo não Encontrado
2,Igor Lorenzo Ramos,Maria Fátima da Costa,MM1043,Rio de Janeiro,Processo Encontrado
3,Iago Thomas Vitor Duarte,Marcela Juliana Vitória Moraes,PC5197,São Paulo,Processo não Encontrado
4,Regina Juliana Carolina Novaes,Tiago Lorenzo Gabriel Alves,PC5030,Distrito Federal,Processo não Encontrado


## Exportar Planilha Atualizada

In [8]:
# Cria uma nova planilha do excel com os dados atualizados
df_processos.to_excel('Processos Atualizados.xlsx')